# Preparação do ambiente

## Bibliotecas

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os.path as osp
import pandas as pd
import re
import unidecode

from datetime import datetime
from joblib import dump

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline

from sklearn.dummy import DummyClassifier

from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier

from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from string import punctuation

In [2]:
import warnings
warnings.filterwarnings("ignore") 

## Constantes e funções auxiliares

In [3]:
FILE_MARKETPLACES = '../datasets/experimento_gs/marketplaces.parquet'
FILE_SUPERVISAO_MERCADO = '../datasets/experimento_gs/supervisao_mercado.xlsx'
FILE_HYPER_PARAMETERS = '../datasets/experimento_gs/gs_hyper_paramenters_classifiers.json'

N_JOBS = 6

In [4]:
def load_file_mercado(file_mercado=FILE_SUPERVISAO_MERCADO):
    dict_df_mercado = pd.read_excel(
    file_mercado,
    sheet_name=None,
    usecols=[2,8,10],
    names=['texto_busca', 'titulo', 'passivel_homologacao'],
    true_values=['Sim', 'sim'],
    false_values=['Não','não'],
    na_values=['-'])

    df_list = []
    for key in dict_df_mercado.keys():
        df = dict_df_mercado[key]
        df['marketplace'] = key
        df_list.append(df)
        
    df_mercado = pd.concat(df_list)
    df_mercado = df_mercado.dropna()
    df_mercado['passivel_homologacao'] = df_mercado['passivel_homologacao'].astype(int)
    
    map_marketplaces = {
        'Amazon': 'Amazon', 
        'Americanas': 'Lojas Americanas',
        'CasasBahia': 'Casas Bahia',
        'Magalu': 'Magazine Luiza', 
        'MercadoLivre': 'Mercado Livre'
    }
    
    df_mercado['marketplace'] = df_mercado['marketplace'].map(map_marketplaces)

    return df_mercado

In [23]:
def save_best_params():
    
    _best_params = {
        'name': gs.best_estimator_['clf'].__class__.__name__,
        'best_params': json.dumps(gs.best_params_),
        'mean_fit_time': gs.cv_results_['mean_fit_time'].mean(),
        'timestamp': datetime.now().timestamp(),
        'train_auc': gs.best_score_,
        'train_accuracy': gs.score(X_test,y_test),
    }

    # test accuracy and roc_auc_score
    test_accuracy = gs.best_estimator_.score(X_test,y_test)
    valid_accuracy = gs.best_estimator_.score(X_valid,y_valid)
    
    if hasattr(gs.best_estimator_['clf'],'predict_proba'):
        y_test_score = gs.best_estimator_.predict_proba(X_test)[:,1]
        y_valid_score = gs.best_estimator_.predict_proba(X_valid)[:,1]
    else:
        y_test_score = gs.best_estimator_.decision_function(X_test)
        y_valid_score = gs.best_estimator_.decision_function(X_valid)
        
    test_auc = roc_auc_score(y_test,y_test_score)
    valid_auc = roc_auc_score(y_valid,y_valid_score)

    _best_params['test_auc'] = test_auc
    _best_params['test_accuracy'] = test_accuracy

    _best_params['valid_auc'] = valid_auc
    _best_params['valid_accuracy'] = valid_accuracy

    # confusion matrix
    y_pred = gs.predict(X_valid)
    cm = confusion_matrix(y_valid,y_pred)
    tn = int(cm[0][0])
    fn = int(cm[1][0])
    tp = int(cm[1][1])
    fp = int(cm[0][1])
    
    confusion = {
        'tn': tn, 
        'fn': fn, 
        'tp': tp, 
        'fp': fp
    }

    _best_params['confusion'] = json.dumps(confusion)
    
    if osp.exists(FILE_HYPER_PARAMETERS):
        with open(FILE_HYPER_PARAMETERS, 'r') as f:
            clf_best_params = json.load(f) 
    else:
        clf_best_params = []
    
    clf_best_params.append(_best_params)
    
    df = pd.DataFrame(clf_best_params)
    df = df.sort_values(by=['test_auc','test_accuracy', 'timestamp'], ascending=False).reset_index(drop=True)
    df = df.drop_duplicates(subset='name',keep='first')

    # save best params
    clf_best_params = df.to_dict('records')
    with open(FILE_HYPER_PARAMETERS, 'w+') as f:
        json.dump(clf_best_params,f,indent=2)

    # save model
    pipe = gs.best_estimator_
    clf_name = pipe['clf'].__class__.__name__
    clf_joblib_file = f'../datasets/experimento_gs/models/{clf_name}.joblib'
    with open(clf_joblib_file,'wb') as f:
        dump(pipe, f)
    
    return df

In [6]:
def load_best_params():

    with open(FILE_HYPER_PARAMETERS) as f:
        clf_best_params = json.load(f)
        
    return pd.DataFrame(clf_best_params)

# Carga e prepação dos dados

In [7]:
df_marketplaces = pd.read_parquet(FILE_MARKETPLACES)
df_mercado = load_file_mercado()

# reduzir a base de dados para construir o notebook
# df_marketplaces = df_marketplaces.sample(10)

# conjunto de dados
docs_marketplaces = df_marketplaces[df_marketplaces['passivel_homologacao']<2]['titulo'].values
targets_marketplaces = df_marketplaces[df_marketplaces['passivel_homologacao']<2]['passivel_homologacao'].values

# Split data to keep experiment results comparable
X_train, X_test, y_train, y_test = train_test_split(
    docs_marketplaces, targets_marketplaces,
    test_size=0.25, 
    random_state=724
)

X_valid, y_valid = df_mercado['titulo'].values, df_mercado['passivel_homologacao'].values

# Análise


https://levity.ai/blog/text-classifiers-in-machine-learning-a-practical-guide

## Ajustando hiperparâmetros dos algoritmos

In [8]:
start_time = datetime.now()

### DummyClassifier

In [24]:
%%time
clf = DummyClassifier()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__strategy": ('most_frequent', 'prior', 'stratified', 'uniform', 'constant'),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

CPU times: total: 406 ms
Wall time: 8.4 s


,name,best_params,mean_fit_time,timestamp,train_auc,train_accuracy,test_auc,test_accuracy,valid_auc,valid_accuracy,confusion
0,DummyClassifier,"{""clf__strategy"": ""most_frequent""}",0.202743,1.727123e+09,0.5,0.5,0.5,0.577374,0.5,0.807329,"{""tn"": 0, ""fn"": 0, ""tp"": 683, ""fp"": 163}"


### NaiveBayes

#### BernoulliNB

In [25]:
%%time
clf = BernoulliNB()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__alpha": np.logspace(-6, 6, 13),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)

save_best_params()

CPU times: total: 422 ms
Wall time: 3.76 s


,name,best_params,mean_fit_time,timestamp,train_auc,train_accuracy,test_auc,test_accuracy,valid_auc,valid_accuracy,confusion
0,BernoulliNB,"{""clf__alpha"": 1.0}",0.226486,1.727123e+09,0.956049,0.953859,0.953859,0.895838,0.949052,0.879433,"{""tn"": 69, ""fn"": 8, ""tp"": 675, ""fp"": 94}"
1,DummyClassifier,"{""clf__strategy"": ""most_frequent""}",0.202743,1.727123e+09,0.500000,0.500000,0.500000,0.577374,0.500000,0.807329,"{""tn"": 0, ""fn"": 0, ""tp"": 683, ""fp"": 163}"


#### ComplementNB

In [11]:
%%time
clf = ComplementNB()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__alpha": np.logspace(-6, 6, 13),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

CPU times: total: 359 ms
Wall time: 3.54 s


,name,best_params,mean_fit_time,timestamp,train_auc,train_accuracy,test_auc,test_accuracy,valid_auc,valid_accuracy,confusion
0,NuSVC,"{""clf__kernel"": ""rbf"", ""clf__nu"": 0.25, ""clf__...",13.869812,1.727113e+09,0.969549,0.971635,0.971635,0.927799,0.957585,0.907801,NaN
1,RandomForestClassifier,"{""clf__criterion"": ""log_loss"", ""clf__max_featu...",62.557479,1.727114e+09,0.969675,0.970364,0.970364,0.913543,0.963716,0.886525,NaN
2,SGDClassifier,"{""clf__alpha"": 1e-05, ""clf__loss"": ""log_loss"",...",0.383845,1.727111e+09,0.967557,0.969969,0.969969,0.917682,0.951944,0.887707,NaN
3,LogisticRegression,"{""clf__C"": 10.0, ""clf__penalty"": ""l2""}",0.342542,1.727111e+09,0.967771,0.969649,0.969649,0.916762,0.955205,0.891253,NaN
4,LinearSVC,"{""clf__C"": 1.0, ""clf__loss"": ""squared_hinge"", ...",0.323706,1.727111e+09,0.966849,0.968172,0.968172,0.918832,0.950561,0.893617,NaN
5,RidgeClassifier,"{""clf__solver"": ""sag""}",6.208579,1.727111e+09,0.966041,0.966691,0.966691,0.915153,0.947731,0.894799,NaN
6,ComplementNB,"{""clf__alpha"": 1.0}",0.218175,1.727122e+09,0.958971,0.957040,0.957040,0.902046,0.954657,0.880615,"{""tn"": 72, ""fn"": 10, ""tp"": 673, ""fp"": 91}"
8,MultinomialNB,"{""clf__alpha"": 1.0}",0.241925,1.727111e+09,0.958971,0.957040,0.957040,0.894229,0.954657,0.875887,NaN
9,KNeighborsClassifier,"{""clf__n_neighbors"": 10, ""clf__p"": 2}",0.262209,1.727111e+09,0.953876,0.956598,0.956598,0.906875,0.897444,0.877069,NaN
10,BernoulliNB,"{""clf__alpha"": 1.0}",0.218894,1.727122e+09,0.956049,0.953859,0.953859,0.895838,0.949052,0.879433,"{""tn"": 69, ""fn"": 8, ""tp"": 675, ""fp"": 94}"


#### MultinomialNB

In [12]:
%%time
clf = MultinomialNB()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__alpha": np.logspace(-6, 6, 13),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe, parameter_grid, scoring=scoring, refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

CPU times: total: 484 ms
Wall time: 3.48 s


,name,best_params,mean_fit_time,timestamp,train_auc,train_accuracy,test_auc,test_accuracy,valid_auc,valid_accuracy,confusion
0,NuSVC,"{""clf__kernel"": ""rbf"", ""clf__nu"": 0.25, ""clf__...",13.869812,1.727113e+09,0.969549,0.971635,0.971635,0.927799,0.957585,0.907801,NaN
1,RandomForestClassifier,"{""clf__criterion"": ""log_loss"", ""clf__max_featu...",62.557479,1.727114e+09,0.969675,0.970364,0.970364,0.913543,0.963716,0.886525,NaN
2,SGDClassifier,"{""clf__alpha"": 1e-05, ""clf__loss"": ""log_loss"",...",0.383845,1.727111e+09,0.967557,0.969969,0.969969,0.917682,0.951944,0.887707,NaN
3,LogisticRegression,"{""clf__C"": 10.0, ""clf__penalty"": ""l2""}",0.342542,1.727111e+09,0.967771,0.969649,0.969649,0.916762,0.955205,0.891253,NaN
4,LinearSVC,"{""clf__C"": 1.0, ""clf__loss"": ""squared_hinge"", ...",0.323706,1.727111e+09,0.966849,0.968172,0.968172,0.918832,0.950561,0.893617,NaN
5,RidgeClassifier,"{""clf__solver"": ""sag""}",6.208579,1.727111e+09,0.966041,0.966691,0.966691,0.915153,0.947731,0.894799,NaN
6,ComplementNB,"{""clf__alpha"": 1.0}",0.218175,1.727122e+09,0.958971,0.957040,0.957040,0.902046,0.954657,0.880615,"{""tn"": 72, ""fn"": 10, ""tp"": 673, ""fp"": 91}"
7,MultinomialNB,"{""clf__alpha"": 1.0}",0.215239,1.727122e+09,0.958971,0.957040,0.957040,0.894229,0.954657,0.875887,"{""tn"": 62, ""fn"": 4, ""tp"": 679, ""fp"": 101}"
9,KNeighborsClassifier,"{""clf__n_neighbors"": 10, ""clf__p"": 2}",0.262209,1.727111e+09,0.953876,0.956598,0.956598,0.906875,0.897444,0.877069,NaN
10,BernoulliNB,"{""clf__alpha"": 1.0}",0.218894,1.727122e+09,0.956049,0.953859,0.953859,0.895838,0.949052,0.879433,"{""tn"": 69, ""fn"": 8, ""tp"": 675, ""fp"": 94}"


### KNeighborsClassifier

In [ ]:
%%time
clf = KNeighborsClassifier()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__n_neighbors": (3, 5, 10),
    "clf__p": (1, 2)
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

### Linear models

#### SGDClassifier

In [ ]:
%%time
clf = SGDClassifier()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__alpha": np.logspace(-6, 6, 13),
    "clf__loss": ('hinge', 'log_loss', 'modified_huber', 'squared_hinge', 'perceptron'),
    "clf__penalty": ('l1', 'l2', 'elasticnet'),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

#### LogisticRegression

In [ ]:
%%time
clf = LogisticRegression(solver='liblinear')

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__C": np.logspace(-5, 5, 11),
    "clf__penalty": ('l1', 'l2'),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

#### RidgeClassifier

In [ ]:
%%time
clf = RidgeClassifier(fit_intercept=False)

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__solver": ('cholesky', 'sparse_cg', 'sag', 'saga'),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

### SVM

#### LinearSVC

In [ ]:
%%time
clf = LinearSVC()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__loss": ('hinge', 'squared_hinge'),
    "clf__C": np.logspace(-5,0,6),
    "clf__tol": np.logspace(-5,0,6),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

#### NuSVC

In [ ]:
%%time
clf = NuSVC()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__nu": (0.1, 0.25, 0.5, 0.75, 1),
    "clf__kernel": ('linear', 'poly', 'rbf', 'sigmoid', 'precomputed'),
    "clf__tol": np.logspace(-5,0,6),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

### Tree

#### DecisionTreeClassifier

In [ ]:
%%time
clf = DecisionTreeClassifier()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__splitter": ('best', 'random'),
    "clf__criterion": ('gini', 'entropy', 'log_loss'),
    "clf__max_features": ('sqrt', 'log2')
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

#### Extremely randomized tree 

In [ ]:
%%time
clf = ExtraTreeClassifier()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__splitter": ('best', 'random'),
    "clf__criterion": ('gini', 'entropy', 'log_loss'),
    "clf__max_features": ('sqrt', 'log2')
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

### Ensemble

#### RandomForestClassifier

In [ ]:
%%time
clf = RandomForestClassifier()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__n_estimators": (10, 100, 500),
    "clf__criterion": ('gini', 'entropy', 'log_loss'),
    "clf__max_features": ('sqrt', 'log2')
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

#### AdaBoostClassifier

In [ ]:
%%time
clf = AdaBoostClassifier()

vectorizer = TfidfVectorizer()
pipe = Pipeline(steps = [('vectorizer',vectorizer),('clf',clf)])

parameter_grid = {
    "clf__n_estimators": (50, 100, 500),
    "clf__learning_rate": np.logspace(-5, 0, 6),
}
scoring = {"AUC": "roc_auc"}

gs = GridSearchCV(pipe,parameter_grid,scoring=scoring,refit='AUC', n_jobs=N_JOBS)
gs.fit(X_train, y_train)
save_best_params()

## Consolidar e exibir resultados

### Tempo de treinamento

In [ ]:
elapsed_time = datetime.now() - start_time
print('Total trainning time: {} seconds'.format(int(elapsed_time.total_seconds())))
print('Total trainning time: {} minutes'.format(int(elapsed_time.total_seconds()//60)))

### Scores

In [ ]:
df = load_best_params()
df

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,5))

# ax.set_xlim(0,1)
# ax.set_ylim(0,1)

for record in df.to_dict('records'):
    label = record['name']
    x = record['test_auc']
    y = record['test_accuracy']
    _=ax.scatter(x,y, label=label)

ax.set_xlabel('Área sob a curva ROC')
ax.set_ylabel('Acurácia')
ax.set_title('Desempenho dos classificadores')
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,5))

# ax.set_xlim(0,1)
# ax.set_ylim(0,1)

for record in df.head(5).to_dict('records'):
    label = record['name']
    x = record['test_auc']
    y = record['test_accuracy']
    _=ax.scatter(x,y, label=label)

ax.set_xlabel('Área sob a curva ROC')
ax.set_ylabel('Acurácia')
ax.set_title('Desempenho dos 5 melhores classificadores')
ax.legend()
plt.show()